In [ ]:
import pandas as pd
import html
import contractions
import unicodedata

In [ ]:
pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 5.4 MB/s eta 0:00:00


In [ ]:
with open('./Data/emojitweets-01-04-2018.txt', 'r') as file: # Downloaded the data file from https://www.kaggle.com/datasets/rexhaif/emojifydata-en
    tweets = file.readlines()

# Create a DataFrame
bigdf = pd.DataFrame(tweets, columns=['tweet'])

In [ ]:
del tweets

In [ ]:
import emoji
import re

def extract_emojis(text): #function to seperate emojis and text
    emojis = emoji.distinct_emoji_list(text)
    text = emoji.replace_emoji(text, replace='')
    text_without_emojis = ' '.join(text.split())
    return text_without_emojis, emojis

In [ ]:
df = bigdf.sample(1000000) # Choosing a random sample of 10M texts

In [ ]:
df[['tweet', 'emojis']] = df['tweet'].apply(lambda x: pd.Series(extract_emojis(x)))

In [ ]:
emojis_array = df.explode('emojis').dropna()['emojis']

In [ ]:
emojis_array.value_counts()[50:100]

emojis
😱        5676
😏        5548
💔        5503
🌹        5496
💋        5430
💘        5410
😈        5401
🖤        5244
☺️       4971
👑        4935
🙃        4900
💦        4863
👇        4828
©        4664
🌸        4625
😌        4624
⚡        4596
💫        4544
😋        4536
💎        4428
😔        4401
😒        4193
📷        3918
😆        3842
➡️       3799
😜        3773
😀        3765
🎂        3722
😤        3704
📸        3637
⭐        3544
💰        3536
🙈        3525
😄        3520
🤷🏽‍♀️    3502
🔴        3465
🤤        3428
⚽        3424
😫        3412
🙂        3324
🤧        3272
❗        3232
😇        3211
🎁        3200
🎥        3127
☀️       3127
🤪        3109
❄️       3005
🙏🏾       2998
🚀        2990
Name: count, dtype: int64

In [ ]:
"""
1. lower case
2. remove html entities
3. remove links
4. remove @
2. remove /
4. convert contractions to single words
2. remove punc
3. remove numbers and symbols
5. lemmatize
observations: html entities are not removed, remove links, remove @
"""

In [ ]:
def preprocess(text):

    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    text = text.lower() # lowercase

    text = html.unescape(text) # remove html entities

    text =  re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # remove links

    text = re.sub(r'@', '', text) # remove @

    text = re.sub(r'/', ' ', text) # convert / to space

    text = contractions.fix(text) # convert contractions to single words

    text = re.sub(r'[^a-z\s]', '', text) # remove punctuations

    return ' '.join(text.split())

In [ ]:
df['processed_text'] = df['tweet'].apply(preprocess)

In [ ]:
top100_emojis = emojis_array.value_counts()[:100].keys().tolist()

In [ ]:
df_exp = df.explode('emojis') # explode the emojis list
df_exp = df_exp[df_exp['emojis'].isin(top100_emojis)] # keep only top 100 emojis
df_final = df_exp.groupby(df_exp.index)['emojis'].apply(list).reset_index() # merge the emojis again
df = df.drop(columns='emojis').merge(df_final, left_index=True, right_on='index')[['processed_text', 'emojis']] # filter the df with new df

In [ ]:
df.to_csv('./Data/processed_tweets.csv', index=False)